In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, Comment
import requests
import re
import html5lib
import lxml
import torch
import torch.nn as nn
from torch import Tensor
from tqdm import tqdm,trange
from torch.utils.data import Dataset
from tqdm import tqdm,trange

In [2]:
from data_helpers.scrape_utils import *
from data_helpers.data_prep import *
from modeling.pytorch_bball import *

In [3]:
year_list = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

advanced_stats = advanced_stats_crawl(year_list)
advanced_stats.head()

Starting: 2010
Starting: 2011
Starting: 2012
Starting: 2013
Starting: 2014
Starting: 2015
Starting: 2016
Starting: 2017
Starting: 2018
Starting: 2019


,player,pos,age,team,games,mp,per,ts,3par,ftr,...,ows,dws,ws,ws48,obpm,dbpm,bpm,vorp,year,player_join
0,Arron Afflalo,SG,24,DEN,82,2221,10.9,.576,.426,.168,...,2.8,1.4,4.3,.092,0.8,-0.5,0.3,1.3,2010,A. Afflalo
1,Alexis Ajinça,C,21,CHA,6,30,6.3,.479,.000,.100,...,-0.1,0.0,0.0,-0.013,-8.5,-2.0,-10.4,-0.1,2010,A. Ajinça
2,LaMarcus Aldridge,PF,24,POR,78,2922,18.2,.535,.014,.260,...,5.5,3.3,8.8,.145,0.9,0.6,1.6,2.6,2010,L. Aldridge
3,Joe Alexander,SF,23,CHI,8,29,2.8,.273,.167,.500,...,0.0,0.0,0.0,.030,-5.2,1.5,-3.7,0.0,2010,J. Alexander
4,Malik Allen,PF,31,DEN,51,456,5.9,.431,.052,.112,...,-0.3,0.3,0.1,.009,-4.2,-0.8,-5.0,-0.3,2010,M. Allen


In [4]:
## Get roster information

## Have to go team by team

team_list = ['TOR', 'BOS', 'DET', 'CHO', 'PHO', 'BRK', 'NYK', 'LAL', 'LAC',
            'MEM', 'DEN', 'WAS', 'HOU', 'GSW', 'PHI', 'UTA', 'MIL', 'IND',
            'CHI', 'CLE', 'SAC', 'POR', 'OKC', 'MIN', 'SAS', 'DAL', 'NOP',
            'MIA', 'ORL', 'ATL']

roster_info = roster_crawl(team_list, year_list)
roster_info

Starting: 2010
Starting: 2011
Starting: 2012
Starting: 2013
Starting: 2014
Starting: 2015
Starting: 2016
Starting: 2017
Starting: 2018
Starting: 2019


,year,player,position,height,weight,bday,country,yrpro
0,2010,Marcus Banks,PG,74,200,"November 19, 1981",us,6
1,2010,Andrea Bargnani,PF,84,245,"October 26, 1985",it,3
2,2010,Marco Belinelli,SG,77,210,"March 25, 1986",it,2
3,2010,Chris Bosh,C,83,235,"March 24, 1984",us,6
4,2010,José Calderón,PG,75,200,"September 28, 1981",es,4
...,...,...,...,...,...,...,...,...
5285,2019,Jordan Sibert,SG,76,187,"August 1, 1992",us,0
5286,2019,Omari Spellman,PF,81,245,"July 21, 1997",us,0
5287,2019,Taurean Waller-Prince,SF,80,220,"March 22, 1994",us,2
5288,2019,Trae Young,PG,74,180,"September 19, 1998",us,0


In [5]:
offsets = ['0', '100', '200', '300', '400', '500', '600', '700', '800', '900', '1000',
          '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000']
    
twoman_data = twoman_crawl(year_list, '200', offsets)

Starting 2010
offset = 0
offset = 100
offset = 200
offset = 300
offset = 400
offset = 500
offset = 600
offset = 700
offset = 800
offset = 900
offset = 1000
offset = 1100
offset = 1200
offset = 1300
offset = 1400
offset = 1500
offset = 1600
offset = 1700
offset = 1800
offset = 1900
offset = 2000
Starting 2011
offset = 0
offset = 100
offset = 200
offset = 300
offset = 400
offset = 500
offset = 600
offset = 700
offset = 800
offset = 900
offset = 1000
offset = 1100
offset = 1200
offset = 1300
offset = 1400
offset = 1500
offset = 1600
offset = 1700
offset = 1800
offset = 1900
offset = 2000
Starting 2012
offset = 0
offset = 100
offset = 200
offset = 300
offset = 400
offset = 500
offset = 600
offset = 700
offset = 800
offset = 900
offset = 1000
offset = 1100
offset = 1200
offset = 1300
offset = 1400
offset = 1500
offset = 1600
offset = 1700
offset = 1800
offset = 1900
offset = 2000
Starting 2013
offset = 0
offset = 100
offset = 200
offset = 300
offset = 400
offset = 500
offset = 600
offset = 

In [7]:
## Write to CSV to avoid going to source each time

twoman_data.to_csv('/Users/Wesley/twoman_netrating/twoman_data.csv', index = False)
roster_info.to_csv('/Users/Wesley/twoman_netrating/roster_data.csv', index = False)
advanced_stats.to_csv('/Users/Wesley/twoman_netrating/advanced_data.csv', index = False)

In [3]:
## Read CSV START HERE

roster_df = pd.read_csv('/Users/Wesley/twoman_netrating/roster_data.csv')
twoman_df = pd.read_csv('/Users/Wesley/twoman_netrating/twoman_data.csv')
advanced_df = pd.read_csv('/Users/Wesley/twoman_netrating/advanced_data.csv')

In [9]:
twoman_df.head()

,rank,player_1,player_2,team,games,minutes,tm_poss,opp_poss,pace,fgm,...,fg_perc,threep,threepa,threepperc,efg,ftm,fta,ftperc,pts,year
0,1,K. Fesenko,D. Williams,UTA,31,203.8,384,387,90.8,13.0,...,0.151,0.8,-7.6,0.186,0.156,-0.1,3.9,-0.085,26.8,2010
1,2,A. Parker,A. Varejão,CLE,75,928.6,1741,1741,90.0,6.2,...,0.070,2.8,2.7,0.090,0.086,5.3,7.6,-0.013,20.5,2010
2,3,K. Fesenko,W. Matthews,UTA,33,210.5,400,396,90.7,10.7,...,0.142,-0.3,-7.4,0.127,0.141,-1.5,4.2,-0.125,19.5,2010
3,4,R. Anderson,D. Howard,ORL,55,495.3,937,926,90.3,4.2,...,0.073,6.5,12.8,0.050,0.114,4.3,6.7,-0.021,19.3,2010
4,5,D. Howard,A. Johnson,ORL,21,204.4,385,385,90.4,2.3,...,0.090,3.9,5.5,0.068,0.121,10.6,15.1,-0.021,19.2,2010


In [4]:
pd.set_option('display.max_columns', 500)
dataset_joiner = dataset_joiner(twoman_df, roster_df, advanced_df)
dataset_joiner.head()

,team,games_base,minutes,tm_poss,opp_poss,pace,fgm,fga,fg_perc,threep,threepa,threepperc,efg,ftm,fta,ftperc,pts,year,player_left,pos_left,age_left,mp_left,per_left,ts_left,3par_left,ftr_left,orb_left,drb_left,trb_left,ast_left,stl_left,blk_left,tov_left,usg_left,ows_left,dws_left,ws_left,ws48_left,obpm_left,dbpm_left,bpm_left,vorp_left,position_left,height_left,weight_left,country_left,yrpro_left,player_right,pos_right,age_right,mp_right,per_right,ts_right,3par_right,ftr_right,orb_right,drb_right,trb_right,ast_right,stl_right,blk_right,tov_right,usg_right,ows_right,dws_right,ws_right,ws48_right,obpm_right,dbpm_right,bpm_right,vorp_right,position_right,height_right,weight_right,country_right,yrpro_right,team_year
0,UTA,31,203.8,384,387,90.8,13.0,-0.6,0.151,0.8,-7.6,0.186,0.156,-0.1,3.9,-0.085,26.8,2010,Kyrylo Fesenko,C,23.0,408.0,9.7,0.533,0.011,0.600,9.8,16.1,13.1,5.3,0.8,3.7,16.7,15.8,0.1,0.5,0.6,0.070,-3.6,0.6,-2.9,-0.1,C,85.0,288.0,ua,2.0,Deron Williams,PG,25.0,2802.0,20.6,0.574,0.246,0.397,2.2,10.4,6.4,44.5,1.8,0.5,16.9,23.8,7.4,3.0,10.3,0.177,4.3,-1.0,3.3,3.7,PG,75.0,200.0,us,4.0,UTA_2010
1,CLE,75,928.6,1741,1741,90.0,6.2,0.6,0.070,2.8,2.7,0.090,0.086,5.3,7.6,-0.013,20.5,2010,Anthony Parker,SG,34.0,2289.0,9.9,0.576,0.549,0.200,1.6,10.0,6.0,9.9,1.4,0.6,12.1,12.0,2.3,2.6,4.9,0.103,0.2,0.6,0.8,1.6,SG,78.0,215.0,us,6.0,Anderson Varejão,PF,27.0,2166.0,15.8,0.598,0.011,0.375,10.9,20.4,15.9,6.1,1.7,2.4,11.0,13.1,4.3,3.8,8.1,0.179,0.4,2.6,3.0,2.7,PF,82.0,273.0,br,5.0,CLE_2010
2,UTA,33,210.5,400,396,90.7,10.7,-2.9,0.142,-0.3,-7.4,0.127,0.141,-1.5,4.2,-0.125,19.5,2010,Kyrylo Fesenko,C,23.0,408.0,9.7,0.533,0.011,0.600,9.8,16.1,13.1,5.3,0.8,3.7,16.7,15.8,0.1,0.5,0.6,0.070,-3.6,0.6,-2.9,-0.1,C,85.0,288.0,ua,2.0,Wesley Matthews,SG,23.0,2025.0,12.3,0.592,0.292,0.342,3.0,8.0,5.6,9.0,1.6,0.6,12.5,16.5,2.6,1.9,4.6,0.108,0.3,-0.5,-0.2,0.9,SG,77.0,220.0,us,0.0,UTA_2010
3,ORL,55,495.3,937,926,90.3,4.2,-4.5,0.073,6.5,12.8,0.050,0.114,4.3,6.7,-0.021,19.3,2010,Ryan Anderson,PF,21.0,911.0,18.1,0.574,0.544,0.211,9.2,16.2,12.8,7.1,1.4,1.1,11.7,24.4,1.7,1.4,3.1,0.161,2.3,-1.0,1.3,0.8,PF,82.0,240.0,us,1.0,Dwight Howard,C,24.0,2843.0,24.0,0.630,0.008,0.978,12.0,31.3,22.0,8.7,1.4,6.0,18.7,23.9,6.1,7.1,13.2,0.223,0.4,5.0,5.5,5.4,C,83.0,265.0,us,5.0,ORL_2010
4,ORL,21,204.4,385,385,90.4,2.3,-11.9,0.090,3.9,5.5,0.068,0.121,10.6,15.1,-0.021,19.2,2010,Dwight Howard,C,24.0,2843.0,24.0,0.630,0.008,0.978,12.0,31.3,22.0,8.7,1.4,6.0,18.7,23.9,6.1,7.1,13.2,0.223,0.4,5.0,5.5,5.4,C,83.0,265.0,us,5.0,Anthony Johnson,PG,35.0,406.0,12.4,0.543,0.351,0.180,2.4,11.0,6.8,24.2,1.4,0.2,17.8,16.7,0.5,0.5,1.0,0.116,-0.6,-0.2,-0.8,0.1,PG,75.0,190.0,us,12.0,ORL_2010


In [5]:
modeling_dataset = dataset_joiner[['player_left', 'player_right','team_year', 'age_left', 'per_left', 'ts_left', '3par_left', 'ftr_left',
                                  'orb_left', 'drb_left', 'trb_left', 'ast_left', 'stl_left', 'blk_left', 'tov_left',
                                  'usg_left', 'ows_left', 'dws_left', 'ws_left', 'ws48_left', 'obpm_left', 'dbpm_left',
                                  'bpm_left', 'bpm_left', 'vorp_left', 'position_left', 'height_left',
                                  'country_left', 'yrpro_left', 'age_right', 'per_right', 'ts_right', '3par_right', 'ftr_right',
                                  'orb_right', 'drb_right', 'trb_right', 'ast_right', 'stl_right', 'blk_right', 'tov_right',
                                  'usg_right', 'ows_right', 'dws_right', 'ws_right', 'ws48_right', 'obpm_right', 'dbpm_right',
                                  'bpm_right', 'bpm_right', 'vorp_right', 'position_right', 'height_right', 
                                  'country_right', 'yrpro_right', 'pts']]
cat_features_emb = [('team_year', 30), ('age_left', 5), ('position_left', 5),
                    ('height_left', 5), ('country_left', 5), ('yrpro_right', 5),
                    ('age_right', 5), ('position_right', 5), ('height_right', 5),
                    ('country_right', 5), ('yrpro_right', 5)]

cat_features = [emb[0] for emb in cat_features_emb]
print(cat_features)
cont_features = list(set(modeling_dataset.columns) - set(cat_features))
cont_features.remove('pts')
cont_features.remove('player_left')
cont_features.remove('player_right')
print(cont_features)
deep_net_cols = cat_features + cont_features
print(deep_net_cols)

['team_year', 'age_left', 'position_left', 'height_left', 'country_left', 'yrpro_right', 'age_right', 'position_right', 'height_right', 'country_right', 'yrpro_right']
['trb_left', '3par_left', 'drb_left', 'blk_right', 'ws_right', 'ws48_right', 'yrpro_left', 'orb_right', '3par_right', 'ast_right', 'obpm_right', 'bpm_left', 'usg_right', 'blk_left', 'ts_right', 'dws_left', 'stl_left', 'dbpm_right', 'ows_left', 'ftr_right', 'per_right', 'dbpm_left', 'trb_right', 'per_left', 'stl_right', 'obpm_left', 'ws_left', 'vorp_right', 'ftr_left', 'dws_right', 'drb_right', 'tov_right', 'usg_left', 'orb_left', 'tov_left', 'ows_right', 'bpm_right', 'vorp_left', 'ts_left', 'ws48_left', 'ast_left']
['team_year', 'age_left', 'position_left', 'height_left', 'country_left', 'yrpro_right', 'age_right', 'position_right', 'height_right', 'country_right', 'yrpro_right', 'trb_left', '3par_left', 'drb_left', 'blk_right', 'ws_right', 'ws48_right', 'yrpro_left', 'orb_right', '3par_right', 'ast_right', 'obpm_right',

In [6]:
train, trainplayers, valid, validplayers, test, testplayers = train_valid_test_splitter(modeling_dataset, 0.8, 888)
train_cats, encoding = cat_encoder(train, cat_features)
valid_cats, encoding = cat_encoder(valid, cat_features, encoding)
test_cats, encoding = cat_encoder(test, cat_features, encoding)

train_base = train_cats.copy()

train_fin, train_y = xandy(min_max(train_cats, train_base, cont_features), 'pts')
valid_fin, valid_y = xandy(min_max(valid_cats, train_base, cont_features), 'pts')
test_fin, test_y = xandy(min_max(test_cats, train_base, cont_features), 'pts')

test_fin.head()

9628
1204
1204
doing this
go
cn
go
go


,team_year,age_left,per_left,ts_left,3par_left,ftr_left,orb_left,drb_left,trb_left,ast_left,stl_left,blk_left,tov_left,usg_left,ows_left,dws_left,ws_left,ws48_left,obpm_left,dbpm_left,bpm_left,bpm_left,vorp_left,position_left,height_left,country_left,yrpro_left,age_right,per_right,ts_right,3par_right,ftr_right,orb_right,drb_right,trb_right,ast_right,stl_right,blk_right,tov_right,usg_right,ows_right,dws_right,ws_right,ws48_right,obpm_right,dbpm_right,bpm_right,bpm_right,vorp_right,position_right,height_right,country_right,yrpro_right
0,46,2,0.739726,0.786301,0.008753,0.798999,0.591133,0.757660,0.805785,0.158700,0.295455,0.60,0.479167,0.548913,0.519337,0.926829,0.714953,0.737789,0.354839,0.889908,0.665072,0.665072,0.580153,1,5,0,0.25,9,0.314286,0.486264,0.384026,0.147662,0.118227,0.192201,0.177686,0.415194,0.300,0.02,0.452381,0.342105,0.209945,0.121951,0.144860,0.490244,0.346734,0.368852,0.338710,0.338710,0.157534,3,5,0,1
1,92,7,0.623288,0.767123,0.002188,0.336113,0.581281,0.618384,0.685950,0.051625,0.159091,0.36,0.291667,0.418478,0.502762,0.353659,0.481308,0.650386,0.338710,0.376147,0.377990,0.377990,0.229008,1,2,44,0.10,6,0.582143,0.543956,0.527352,0.349467,0.068966,0.089136,0.074380,0.227915,0.525,0.04,0.172619,0.613158,0.480663,0.195122,0.406542,0.590244,0.577889,0.131148,0.407258,0.407258,0.253425,4,6,0,11
2,39,19,0.428082,0.345205,0.459519,0.193495,0.113300,0.228412,0.190083,0.357553,0.363636,0.05,0.202381,0.774457,0.132597,0.121951,0.079439,0.138817,0.376344,0.128440,0.282297,0.282297,0.152672,4,14,0,0.95,14,0.503571,0.598901,0.505470,0.502051,0.083744,0.111421,0.095041,0.254417,0.350,0.08,0.244048,0.502632,0.430939,0.097561,0.317757,0.497561,0.527638,0.163934,0.383065,0.383065,0.226027,4,6,0,15
3,163,11,0.445205,0.432877,0.003282,0.222686,0.443350,0.495822,0.537190,0.133843,0.090909,0.51,0.273810,0.478261,0.237569,0.451220,0.294393,0.434447,0.166667,0.623853,0.354067,0.354067,0.206107,1,15,0,0.30,13,0.442857,0.390110,0.031729,0.192781,0.315271,0.434540,0.438017,0.342756,0.275,0.20,0.330357,0.455263,0.259669,0.402439,0.299065,0.470732,0.341709,0.581967,0.439516,0.439516,0.273973,0,2,0,14
4,21,12,0.366438,0.441096,0.487965,0.143453,0.147783,0.415042,0.359504,0.131931,0.522727,0.31,0.166667,0.448370,0.193370,0.475610,0.266355,0.411311,0.258065,0.587156,0.416268,0.416268,0.244275,1,3,0,0.70,5,0.375000,0.675824,0.008753,0.664479,0.487685,0.437326,0.516529,0.081272,0.225,0.28,0.383929,0.305263,0.243094,0.170732,0.191589,0.602439,0.276382,0.467213,0.326613,0.326613,0.157534,0,2,0,2


In [7]:
embeddings_input = []
emb_dict = dict(cat_features_emb)
for k,v in encoding.items():
    embeddings_input.append((k, len(v), emb_dict[k]))
deep_column_idx = {k:v for v,k in enumerate(train_fin.columns)}

In [14]:
model_dd = DeepDense(embeddings_input = embeddings_input,
                   embeddings_encoding_dict = emb_dict,
                   continuous_cols = cont_features,
                   deep_column_idx = deep_column_idx,
                   hidden_layers = [64,16],
                   dropout = [0.1],
                   output_dim = 1
                  )

116
[116, 64, 16]


In [15]:
model_dd.model_compile(method='regression', optimizer=['Adam', 0.01], lr_scheduler=['StepLR', 3, 0.01])
use_cuda = torch.cuda.is_available()
if use_cuda:
    model1 = model1.cuda()
use_cuda

False

In [16]:
traindataset = model_dataloader(train_fin.values, train_y, mode = 'train', uhlog = False)
validdataset = model_dataloader(valid_fin.values, valid_y, mode = 'train', uhlog = False)
testdataset = model_dataloader(test_fin.values, test_y, mode = 'test', uhlog = False)

train_loader = torch.utils.data.DataLoader(dataset=traindataset, batch_size=64,shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=validdataset, batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testdataset, batch_size=64,shuffle=False)

model_dd.fit(n_epochs=10, train_loader=train_loader, eval_loader=valid_loader)

/Users/Wesley/anaconda3/envs/py37/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
valid: 100%|██████████| 19/19 [00:00<00:00, 92.79it/s, loss=5.33]


In [17]:
from sklearn.metrics import mean_squared_error

preds = model_dd.predict(test_loader)
print(np.sqrt(mean_squared_error(test_y, preds)))

predict: 100%|██████████| 19/19 [00:00<00:00, 162.03it/s]

5.552931996157013


In [18]:
testplayers = testplayers.reset_index(drop = True)
testplayers['actual'] = test_y
testplayers['prediction'] = preds
testplayers['delta'] = test_y - preds
print(max(preds))
print(min(preds))
testplayers.sort_values(by='prediction', ascending = False).head(n=50)

14.3443365
-15.692449


,player_left,player_right,team_year,actual,prediction,delta
458,James Harden,Chris Paul,HOU_2018,13.7,14.344337,-0.644337
95,Harrison Barnes,Stephen Curry,GSW_2016,16.3,14.093141,2.206859
1047,Nick Collison,Kevin Durant,OKC_2014,18.5,14.056699,4.443301
843,Kevin Durant,Kevin Martin,OKC_2013,11.8,13.869270,-2.069270
1149,Stephen Curry,Klay Thompson,GSW_2015,17.8,13.641061,4.158939
835,Chris Andersen,LeBron James,MIA_2013,14.7,13.256126,1.443874
24,Stephen Curry,Andre Iguodala,GSW_2017,18.1,13.114052,4.985948
6,Zydrunas Ilgauskas,LeBron James,CLE_2010,15.6,12.820673,2.779327
100,Steven Adams,Russell Westbrook,OKC_2016,13.0,12.457008,0.542992
18,Kevin Durant,Zaza Pachulia,GSW_2017,21.7,12.203784,9.496216


## To Do List

1. Pull data across all 3 datasets for full time frame DONE
2. Join data across all 3 datasets - evaluate loss for join with two man due to naming convention DONE
3. Clean out + & - in twoman data DONE
4. Save resulting data to CSV/S3. Save in files per year in future, for now just big file DONE
5. Begin processing data for Modeling process
    a. One-hot encode categorical variables DONE
    b. minmax scale continuous features (values between 0 & 1) DONE
    c. Possible adds - left & right are arbitrary. Mix to create canonical total? Keep consistent list of min max vals?
6. Begin modeling process
7. Look at https://github.com/jrzaurin/Wide-and-Deep-PyTorch/blob/master/widedeep/models/wide_deep.py to see how to execute model. Need to set optimizer etc...
    a. Wide & deep networks:
        1. https://data-flair.training/blogs/tensorflow-wide-and-deep-learning/
        2. https://arxiv.org/pdf/1606.07792.pdf
        3. Look at links in deep learning folder for more detail
        
8. Experiment with model 
9. Figure out how to convert prediction back to actual value DONE - figure out how to handle test data properly - DONE!! but need to figure out how to handle missing data now just moving to 0. 
10. Convert to SM with cuda - think that's causing predict error.
10. Evaluate impact



In [ ]:
def ds_creator(df, year):
    
    ds = df[df['year'] == year]
    
    return ds

ds_2010 = ds_creator(dataset_joiner, '2010')
ds_2011 = ds_creator(dataset_joiner, '2011')
ds_2012 = ds_creator(dataset_joiner, '2012')
ds_2013 = ds_creator(dataset_joiner, '2013')
ds_2014 = ds_creator(dataset_joiner, '2014')
ds_2015 = ds_creator(dataset_joiner, '2015')
ds_2016 = ds_creator(dataset_joiner, '2016')
ds_2017 = ds_creator(dataset_joiner, '2017')
ds_2018 = ds_creator(dataset_joiner, '2018')
ds_2019 = ds_creator(dataset_joiner, '2019')

ds_2010.to_csv('/Users/Wesley/bball/dataset_2010.csv', index = False)
ds_2011.to_csv('/Users/Wesley/bball/dataset_2011.csv', index = False)
ds_2012.to_csv('/Users/Wesley/bball/dataset_2012.csv', index = False)
ds_2013.to_csv('/Users/Wesley/bball/dataset_2013.csv', index = False)
ds_2014.to_csv('/Users/Wesley/bball/dataset_2014.csv', index = False)
ds_2015.to_csv('/Users/Wesley/bball/dataset_2015.csv', index = False)
ds_2016.to_csv('/Users/Wesley/bball/dataset_2016.csv', index = False)
ds_2017.to_csv('/Users/Wesley/bball/dataset_2017.csv', index = False)
ds_2018.to_csv('/Users/Wesley/bball/dataset_2018.csv', index = False)
ds_2019.to_csv('/Users/Wesley/bball/dataset_2019.csv', index = False)